In [1]:
import pandas as pd
from PIL import Image
import os
import pytesseract
import re
df=pd.read_csv('dataset/test.csv')

In [2]:
df.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [3]:
df.dtypes

index           int64
image_link     object
group_id        int64
entity_name    object
dtype: object

In [4]:
from pathlib import Path

In [5]:
df['image_name']=df['image_link'].apply(lambda link:Path(link).name)

In [6]:
df.head()

,index,image_link,group_id,entity_name,image_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,110EibNyclL.jpg
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg


In [7]:
# Given unit_suffix_map
unit_suffix_map = {
    'centimetre': {'cm', 'centimeter', 'centimetre'},
    'metre': {'m', 'meter', 'metre'},
    'millimetre': {'mm', 'millimeter', 'millimetre'},
    'foot': {'ft', 'foot'},
    'inch': {'in', 'inch'},
    'yard': {'yd', 'yard'},
    'gram': {'g', 'gram'},
    'kilogram': {'kg', 'kilogram'},
    'microgram': {'μg', 'microgram', 'mcg'},
    'milligram': {'mg', 'milligram'},
    'ounce': {'oz', 'ounce'},
    'pound': {'lb', 'pound'},
    'ton': {'t', 'ton'},
    'kilovolt': {'kV', 'kilovolt'},
    'millivolt': {'mV', 'millivolt'},
    'volt': {'V', 'volt'},
    'kilowatt': {'kW', 'kilowatt'},
    'watt': {'W', 'watt'},
    'centilitre': {'cl', 'centilitre', 'centiliter'},
    'cubic foot': {'cu ft', 'cubic foot'},
    'cubic inch': {'cu in', 'cubic inch'},
    'cup': {'cup'},
    'decilitre': {'dl', 'decilitre', 'deciliter'},
    'fluid ounce': {'fl oz', 'fluid ounce'},
    'gallon': {'gal', 'gallon'},
    'imperial gallon': {'imp gal', 'imperial gallon'},
    'litre': {'L', 'litre', 'liter'},
    'microlitre': {'μL', 'microlitre', 'microliter'},
    'millilitre': {'mL', 'millilitre', 'milliliter'},
    'pint': {'pt', 'pint'},
    'quart': {'qt', 'quart'}
}

In [8]:
def extract_values_and_units(text, unit_suffix_map):
    results = []

    # Create a regex pattern to find numbers followed by potential unit suffixes
    for unit, suffixes in unit_suffix_map.items():
        for suffix in suffixes:
            # Regex pattern to match a number followed by the unit (with optional space)
            pattern = r'(\d+\.?\d*)\s*(' + re.escape(suffix) + r')'
            matches = re.findall(pattern, text, re.IGNORECASE)
            
            # Append matched number and unit
            for match in matches:
                number = match[0]  # The number
                found_unit = unit  # The full unit name
                results.append((number, found_unit))

    return results

In [9]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}


In [10]:
# Traverse through the directory

def final(entity_name1,image_name1):
    directory_path='test_images/'
    allowed_units=list(entity_unit_map[entity_name1])
    file_path = os.path.join(directory_path, image_name1)
    extracted_text=pytesseract.image_to_string(Image.open(file_path))
    extracted_values_units = extract_values_and_units(extracted_text, unit_suffix_map)
    if extracted_values_units:
        for number, unit in extracted_values_units:
                for i in allowed_units:
                     if unit==i:
                          xgf=round(float(number),1)
                          s=str(xgf)+' '+unit
                          return s 
                          
    else:
        return "No units and values found."
        


In [12]:
# Replace 'train_images' with your actual folder path
folder_path = 'test_images'


# Get a list of image filenames in the folder
existing_images = os.listdir(folder_path)

# Filter the DataFrame based on image existence
df2 = df[df['image_name'].isin(existing_images)]

In [13]:
df2

,index,image_link,group_id,entity_name,image_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,110EibNyclL.jpg
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg
...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,A1rVsIzEtkL.jpg
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,A1rdvZ5zDdL.jpg
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,A1rdvZ5zDdL.jpg
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,A1tnTUPyr7L.jpg


In [14]:
df2['predicted_entity_value'] = None

# Apply the final function to each row with existing images
for index, row in df2.iterrows():
    image_name = row['image_name']
    entity_name = row['entity_name']

    if image_name in existing_images:  # Check if image exists
        try:
            predicted_value = final(entity_name, image_name)
            df2.at[index, 'predicted_entity_value'] = predicted_value
        except Exception as e:  # Handle potential errors during image processing
            print(f"Error processing image {image_name}: {e}")

In [15]:
df2

,index,image_link,group_id,entity_name,image_name,predicted_entity_value
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,110EibNyclL.jpg,No units and values found.
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg,42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg,42.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg,10.5 centimetre
...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,A1rVsIzEtkL.jpg,No units and values found.
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,A1rdvZ5zDdL.jpg,No units and values found.
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,A1rdvZ5zDdL.jpg,No units and values found.
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,A1tnTUPyr7L.jpg,No units and values found.


In [16]:
df2.to_csv('generated_test.csv',index=False)

In [17]:
df2['predicted_entity_value'].notnull()

0         True
1         True
2         True
3         True
4         True
          ... 
131182    True
131183    True
131184    True
131185    True
131186    True
Name: predicted_entity_value, Length: 131187, dtype: bool

In [18]:
df3=df2[df2['predicted_entity_value'].notnull()]

In [19]:
df3

,index,image_link,group_id,entity_name,image_name,predicted_entity_value
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,110EibNyclL.jpg,No units and values found.
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg,42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg,42.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg,10.5 centimetre
...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,A1rVsIzEtkL.jpg,No units and values found.
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,A1rdvZ5zDdL.jpg,No units and values found.
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,A1rdvZ5zDdL.jpg,No units and values found.
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,A1tnTUPyr7L.jpg,No units and values found.


In [20]:
df3

,index,image_link,group_id,entity_name,image_name,predicted_entity_value
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,110EibNyclL.jpg,No units and values found.
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg,42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg,42.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg,10.5 centimetre
...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,A1rVsIzEtkL.jpg,No units and values found.
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,A1rdvZ5zDdL.jpg,No units and values found.
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,A1rdvZ5zDdL.jpg,No units and values found.
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,A1tnTUPyr7L.jpg,No units and values found.


In [21]:
df4=df3[~(df3['predicted_entity_value']=='No units and values found.')]

In [22]:
df4

,index,image_link,group_id,entity_name,image_name,predicted_entity_value
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg,42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg,42.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg,10.5 centimetre
5,5,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,height,11gHj8dhhrL.jpg,10.5 centimetre
...,...,...,...,...,...,...
131157,131258,https://m.media-amazon.com/images/I/91zrylGhoR...,749917,item_weight,91zrylGhoRL.jpg,40.0 pound
131164,131265,https://m.media-amazon.com/images/I/A1EzG3doSg...,941694,item_volume,A1EzG3doSgL.jpg,11.5 fluid ounce
131168,131269,https://m.media-amazon.com/images/I/A1OFrK2oyN...,625842,item_volume,A1OFrK2oyNL.jpg,16.0 fluid ounce
131169,131270,https://m.media-amazon.com/images/I/A1Q-HkaBMN...,941694,item_volume,A1Q-HkaBMNL.jpg,414.0 millilitre


In [23]:
df4.dtypes

index                      int64
image_link                object
group_id                   int64
entity_name               object
image_name                object
predicted_entity_value    object
dtype: object

In [25]:
df2[df2.duplicated(subset='image_link',keep=False)]

,index,image_link,group_id,entity_name,image_name,predicted_entity_value
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg,42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg,42.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg,10.5 centimetre
5,5,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,height,11gHj8dhhrL.jpg,10.5 centimetre
...,...,...,...,...,...,...
131166,131267,https://m.media-amazon.com/images/I/A1KuAtMexv...,603688,maximum_weight_recommendation,A1KuAtMexvL.jpg,No units and values found.
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,A1rdvZ5zDdL.jpg,No units and values found.
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,A1rdvZ5zDdL.jpg,No units and values found.
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,A1tnTUPyr7L.jpg,No units and values found.


In [26]:
df2 = df2.rename(columns={'predicted_entity_value': 'prediction'})

In [27]:
df2

,index,image_link,group_id,entity_name,image_name,prediction
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,110EibNyclL.jpg,No units and values found.
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg,42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg,42.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg,10.5 centimetre
...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,A1rVsIzEtkL.jpg,No units and values found.
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,A1rdvZ5zDdL.jpg,No units and values found.
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,A1rdvZ5zDdL.jpg,No units and values found.
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,A1tnTUPyr7L.jpg,No units and values found.


In [28]:
df2.dtypes

index           int64
image_link     object
group_id        int64
entity_name    object
image_name     object
prediction     object
dtype: object

In [29]:
# Replace NaN values with an empty string
df2['prediction'] = df2['prediction'].fillna('')

# Replace 'No units and values found.' with an empty string
df2['prediction'] = df2['prediction'].replace('No units and values found.', '')

# Ensure the column is of string type
df2['prediction'] = df2['prediction'].astype(str)

index           int64
image_link     object
group_id        int64
entity_name    object
image_name     object
prediction     object
dtype: object

In [32]:
df3=df2[df2['prediction'].notnull()]

In [33]:
df3

,index,image_link,group_id,entity_name,image_name,prediction
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,110EibNyclL.jpg,
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,11TU2clswzL.jpg,42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,11TU2clswzL.jpg,42.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,11TU2clswzL.jpg,42.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,11gHj8dhhrL.jpg,10.5 centimetre
...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,A1rVsIzEtkL.jpg,
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,A1rdvZ5zDdL.jpg,
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,A1rdvZ5zDdL.jpg,
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,A1tnTUPyr7L.jpg,


In [35]:
X=df2[['index', 'prediction']]

In [36]:
df4=df3[(df3['prediction']=='')]

In [37]:
df4

,index,image_link,group_id,entity_name,image_name,prediction
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,110EibNyclL.jpg,
7,7,https://m.media-amazon.com/images/I/11lshEUmCr...,156839,height,11lshEUmCrL.jpg,
10,10,https://m.media-amazon.com/images/I/213oP6n7jt...,953313,item_weight,213oP6n7jtL.jpg,
11,11,https://m.media-amazon.com/images/I/213wY3gUsm...,276611,width,213wY3gUsmL.jpg,
12,12,https://m.media-amazon.com/images/I/214CLs1ozn...,648011,depth,214CLs1oznL.jpg,
...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,A1rVsIzEtkL.jpg,
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,A1rdvZ5zDdL.jpg,
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,A1rdvZ5zDdL.jpg,
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,A1tnTUPyr7L.jpg,


In [38]:
X.to_csv('output.csv',index=False)